#### Albans

In [ ]:
# ─── IMPORTS NECESARIOS ─────────────────────────────────
from pathlib import Path
from datetime import datetime
from typing import List, Dict
import torch
import os
import moviepy.editor as mp
from faster_whisper import WhisperModel
from whisperx.diarize import DiarizationPipeline
import whisperx
from docx import Document

# ─── PARÁMETROS (cámbialos) ─────────────────────────────
# ─── PARÁMETROS ───────────────────────────────────────────
VIDEO       = r"RUTA_VIDEO" 
WORK_DIR    = Path(r"RUTA_DESTINO_OUTPUTS")
MODEL_SIZE  = "small"                     # tiny | small | medium | large
# PREVIEW_SEC = 300                         # 300 = 5 min · 600 = 10 min
HF_TOKEN    = os.getenv("HF_TOKEN") or "TU_TOKEN_DE_HUGGING_FACE"   # None si offline

class LocalWhisperTranscriber:
    """
    Clase para transcribir videos usando Whisper + WhisperX,
    con soporte para exportación a texto o DOCX con diarización de hablantes.
    """

    def __init__(self, video_path, model_size="small", work_dir="outputs", device=None):
        self.video_path = Path(video_path)
        self.work_dir   = Path(work_dir); self.work_dir.mkdir(exist_ok=True)
        self.audio_path = None  # Se asigna luego por extract_audio_full() o extract_audio_preview()
        self.model_size = model_size
        self.device     = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.transcript: List[Dict] = []

    # --- Extrae TODO el audio completo del video ----------
    def extract_audio_full(self, sr=16_000) -> Path:
        """
        Extrae todo el audio del video en formato .wav mono, 16kHz.
        Guarda en: nombre_video_16k.wav
        """
        out = self.work_dir / f"{self.video_path.stem}_16k.wav"
        mp.VideoFileClip(str(self.video_path)).audio.write_audiofile(
            str(out), fps=sr, codec="pcm_s16le", nbytes=2, ffmpeg_params=["-ac", "1"])
        self.audio_path = out
        return out

    # --- Extrae SOLO una previsualización en segundos -----
    def extract_audio_preview(self, seconds=300, sr=16_000) -> Path:
        """
        Extrae solo los primeros `seconds` segundos del video.
        Útil para pruebas rápidas o resumen de los primeros minutos.
        """
        out = self.work_dir / f"{self.video_path.stem}_preview_{seconds}s.wav"
        mp.VideoFileClip(str(self.video_path)).subclip(0, seconds).audio.write_audiofile(
            str(out), fps=sr, codec="pcm_s16le", nbytes=2, ffmpeg_params=["-ac", "1"])
        self.audio_path = out
        return out

    # --- TRANSCRIPCIÓN DE AUDIO ---------------------------
    def transcribe(self, beam_size=5, vad_filter=True):
        """
        Transcribe el audio actual (definido por self.audio_path) usando Faster-Whisper.
        Requiere haber ejecutado previamente extract_audio_full o extract_audio_preview.
        """
        if not self.audio_path or not self.audio_path.exists():
            raise FileNotFoundError("⚠️ No se encontró el archivo de audio. Ejecuta primero extract_audio_full() o extract_audio_preview().")

        compute = "float16" if self.device == "cuda" else "int8_float32"
        model = WhisperModel(self.model_size, device=self.device, compute_type=compute)
        segments, _ = model.transcribe(
            str(self.audio_path),
            beam_size=beam_size, vad_filter=vad_filter, word_timestamps=True)
        self.transcript = [
            {"start": s.start, "end": s.end, "text": s.text.strip()} for s in segments]
        return self.transcript

    # --- DIARIZACIÓN CON WHISPERX -------------------------
    def diarize(self, speakers: int | None = None,
                hf_token: str | None = None,
                model_name: str = "pyannote/speaker-diarization-3.1"):
        """
        Etiqueta cada fragmento con el hablante correspondiente usando WhisperX + pyannote.
        """
        pipe = DiarizationPipeline(device=self.device,
                                   model_name=model_name,
                                   use_auth_token=hf_token)
        if speakers:
            pipe.num_speakers = speakers

        diar = pipe(str(self.audio_path))
        self.transcript = whisperx.assign_word_speakers(
            diar, {"segments": self.transcript})["segments"]
        return self.transcript

    # --- EXPORTA TRANSCRIPCIÓN ----------------------------
    def export(self, fmt="docx", include_speakers=True) -> Path:
        """
        Exporta la transcripción a DOCX o TXT.
        Incluye marcas de tiempo y nombres de hablantes (si están disponibles).
        """
        ts  = datetime.now().strftime("%Y%m%d_%H%M%S")
        out = self.work_dir / f"{self.video_path.stem}_{ts}.{fmt}"
        hms = lambda s: f"{int(s//3600):02}:{int(s%3600//60):02}:{int(s%60):02}"

        if fmt == "txt":
            with out.open("w", encoding="utf-8") as f:
                for seg in self.transcript:
                    spk = f"{seg['speaker']}: " if include_speakers and 'speaker' in seg else ""
                    f.write(f"[{hms(seg['start'])}-{hms(seg['end'])}] {spk}{seg['text']}\n")

        elif fmt == "docx":
            doc = Document(); doc.add_heading(self.video_path.name, level=1)
            doc.add_paragraph(f"Generado {datetime.now():%d-%b-%Y %H:%M}")
            for seg in self.transcript:
                p = doc.add_paragraph()
                if include_speakers and 'speaker' in seg:
                    p.add_run(f"{seg['speaker']} ").italic = True
                p.add_run(f"{hms(seg['start'])}-{hms(seg['end'])} ").bold = True
                p.add_run(seg['text'])
            doc.save(out)
        else:
            raise ValueError("Formato no soportado: usa 'txt' o 'docx'")

        return out



Ejecución:

In [ ]:
tr = LocalWhisperTranscriber(VIDEO, MODEL_SIZE, WORK_DIR)

tr.extract_audio_full()                        # 🎙️ Extrae TODO el audio
tr.transcribe()                                # ✍️ Transcribe
tr.diarize(speakers=2, hf_token=HF_TOKEN)      # 🧍 Etiqueta los hablantes
docx = tr.export("docx")                       # 📄 Exporta el DOCX
print("✅ DOCX generado:", docx)

Código para anonimizar información y utilizar transcripciones con LLM's en la nube de manera segura y sin comprometer la privacidad:

In [ ]:
import docx
import re # Importamos la librería de expresiones regulares para reemplazos más potentes

def anonimizar_word(ruta_entrada, ruta_salida, diccionario_reemplazos):
    """
    Carga un documento de Word, reemplaza el texto sensible según un diccionario
    y guarda el resultado en un nuevo archivo.

    Args:
        ruta_entrada (str): La ruta al archivo .docx original.
        ruta_salida (str): La ruta donde se guardará el nuevo archivo .docx modificado.
        diccionario_reemplazos (dict): Un diccionario donde las claves son las palabras a buscar
                                      y los valores son las palabras de reemplazo.
    """
    try:
        # Cargar el documento de Word
        documento = docx.Document(ruta_entrada)
        
        print("Iniciando proceso de anonimización...")
        print(f"Cargando documento: {ruta_entrada}")

        # La lógica de reemplazo debe iterar sobre párrafos y tablas.

        # 1. Reemplazar en los párrafos del cuerpo del documento
        for parrafo in documento.paragraphs:
            for clave, valor in diccionario_reemplazos.items():
                # Usamos re.sub() para reemplazar todas las ocurrencias en el texto del párrafo
                # Esto es más robusto que un simple .replace()
                if re.search(clave, parrafo.text):
                    # Guardamos el texto original del párrafo para no perderlo
                    texto_original = parrafo.text
                    # Realizamos el reemplazo
                    texto_modificado = re.sub(clave, valor, texto_original)
                    
                    # Limpiamos el párrafo original y añadimos el nuevo texto
                    # Esto ayuda a mantener parte del formato, aunque puede ser complejo.
                    # Una forma simple es reemplazar el texto completo del párrafo.
                    # Para reemplazos complejos que mantienen el formato (negritas, cursivas),
                    # se debe iterar sobre los 'runs' del párrafo, lo cual es más avanzado.
                    # Por simplicidad y efectividad, este enfoque reemplaza el texto completo del párrafo.
                    if texto_original != texto_modificado:
                         parrafo.text = texto_modificado


        # 2. Reemplazar en las tablas del documento
        for tabla in documento.tables:
            for fila in tabla.rows:
                for celda in fila.cells:
                    for parrafo in celda.paragraphs:
                        for clave, valor in diccionario_reemplazos.items():
                            if re.search(clave, parrafo.text):
                                texto_original = parrafo.text
                                texto_modificado = re.sub(clave, valor, texto_original)
                                if texto_original != texto_modificado:
                                    parrafo.text = texto_modificado

        # Guardar el documento modificado
        documento.save(ruta_salida)
        print("-" * 30)
        print(f"¡Proceso completado! Documento anonimizado guardado en: {ruta_salida}")

    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta '{ruta_entrada}'")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

# --- CONFIGURACIÓN Y EJECUCIÓN ---
if __name__ == "__main__":
    # 1. Define la ruta de tu documento original
    archivo_original = r"RUTA_DE_LA_TRANSCRIPCIÓN"

    # 2. Define la ruta para el nuevo documento (el resultado)
    archivo_anonimizado = r"RUTA_DE_SALIDA_DE_LA_TRANSCRIPCIÓN_BLINDADA"

    # 3. DEFINE AQUÍ TODAS LAS PALABRAS QUE QUIERES CAMBIAR
    # Clave: La palabra o patrón a buscar.
    # Valor: El texto por el que se reemplazará.
    mapa_de_reemplazos = {
        "NOMBRE_EMPRESA": "[ORG]",
        "NOMBRE_PERSONA": "[PERSON]",
        # # Ejemplo con expresión regular para encontrar cualquier DNI/ID con formato similar
        # r'\d{8}[A-Z]': '[DNI_REDACTADO]',
        # # Ejemplo para correos electrónicos
        # r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b': '[EMAIL_REDACTADO]'
    }

    # Llamar a la función para realizar el proceso
    anonimizar_word(archivo_original, archivo_anonimizado, mapa_de_reemplazos)

#### Cortes de video

Las transcripciones vienen con el timer de la conversación; si se manda el documento blindado a NotebookLM (Suite de Google AI & Gemini) Permite consultarle momentos y hallazgos más importantes y extraer el pedazo de video para presentaciones o para almacenar evidencia.

In [ ]:
import moviepy.editor as mp
from pathlib import Path

def cortar_video(video_path: str, output_dir: str, start_time: float, end_time: float, output_name: str):
    """
    Extrae y guarda un fragmento (clip) de un video.

    Args:
        video_path (str): Ruta al video original.
        output_dir (str): Directorio donde se guardarán los clips.
        start_time (float): Tiempo de inicio del clip en segundos.
        end_time (float): Tiempo de fin del clip en segundos.
        output_name (str): Nombre del archivo de video de salida (sin extensión).
    """
    # Asegurarse de que el directorio de salida exista
    Path(output_dir).mkdir(exist_ok=True)
    
    output_path = Path(output_dir) / f"{output_name}.mp4"

    print(f"🎬 Procesando clip: '{output_name}.mp4'")
    print(f"   - Intervalo: desde {start_time:.2f}s hasta {end_time:.2f}s")

    try:
        # Usar un bloque 'with' para asegurar que los recursos del video se liberen
        with mp.VideoFileClip(video_path) as video:
            # Crear el subclip con el intervalo de tiempo especificado
            clip = video.subclip(start_time, end_time)
            
            # Escribir el nuevo archivo de video con codecs optimizados para web
            # H.264 para video y AAC para audio son universalmente compatibles.
            clip.write_videofile(
                str(output_path), 
                codec="libx264", 
                audio_codec="aac",
                temp_audiofile='temp-audio.m4a', 
                remove_temp=True,
                logger='bar' # Muestra una barra de progreso
            )
        
        print(f"✅ ¡Clip '{output_name}.mp4' guardado con éxito!\n")

    except Exception as e:
        print(f"❌ Error al procesar el clip '{output_name}.mp4': {e}\n")


# --- CONFIGURACIÓN PRINCIPAL ----------------------------------------------------
if __name__ == "__main__":
    # --- 1. Define tu video y dónde guardar los clips ---
    VIDEO_ORIGINAL = r"RUTA_VIDEO_ORIGINAL"
    DIRECTORIO_SALIDA = r"RUTA_OUTPUT"

    # --- 2. Define la lista de clips que quieres extraer ---
    # Para cada clip, define:
    #   'inicio': En segundos. Puedes calcularlo (minutos * 60 + segundos).
    #   'fin': En segundos.
    #   'nombre': El nombre que tendrá el archivo de salida.
    # clips_a_extraer = [
    #     {
    #         "nombre": "testimonio_clienta_1",
    #         "inicio": (26 * 60) + 18,  # 2m 10s
    #         "fin": (27 * 60) + 18       # 3m 05s
    #     }
    # ]
    
    clips_a_extraer = [
        {
            "nombre": "testimonio_cliente_2",
            "inicio": (50 * 60) + 3,  # 2m 10s
            "fin": (51 * 60) + 3       # 3m 05s
        }
    ]
    
    # clips_a_extraer = [
    #     {
    #         "nombre": "testimonio_cliente_3",
    #         "inicio": (6 * 60) + 48,  #
    #         "fin": (7 * 60) + 50     # 
    #     }
    # ]

    print("--- INICIANDO PROCESO DE EXTRACCIÓN DE CLIPS ---\n")

    # --- 3. El script procesará cada clip de la lista ---
    for clip_info in clips_a_extraer:
        cortar_video(
            video_path=VIDEO_ORIGINAL,
            output_dir=DIRECTORIO_SALIDA,
            start_time=clip_info["inicio"],
            end_time=clip_info["fin"],
            output_name=clip_info["nombre"]
        )

    print("🎉 --- ¡Proceso completado! Todos los clips han sido extraídos. ---")